### KOA archive access examples

This notebook presents the examples of running the Keck On-line-Archive (KOA)'s python interface for accessing KOA archive, the code is in GitHub: https://github.com/Caltech-IPAC/KOA_TAP_Client/ .  This document is not meant to server as a user interface, but an example to illustrate how to run the KOA's client interface.  The Archive class in 'koapy' package consists of three main types of fuctions: login method (for user login), request methods (for accessing KOA's metadata), and download method (for downloading the files in the metadata tables).

##### <font color="#880000">Note: plateform requirement</font> 
koapy requires the installation of python3.6 (or above), plus astropy package.

##### <font color="#880000">Note: Input parameter types</font> 
Without specifically mentioning in the examples, all the "position input parameters" in the examples are required, and all the "keyword parameters" are optional input parameters. 

#### Login

Login is an optional step, only needed for users who has KOA accounts to access their proprietary data. 

The login class requires a cookie file input to save the return cookie which will be the required input for the subsequent proprietary data access.

If the keyword parameters 'userid' and 'password' are not provided in the login call, the program will prompt for userid and password and upon entering the correct inputs, it will report the login status as shown in the following example:


In [1]:
import archive

arch = archive.Archive ()

archive class initialized


In [2]:
arch.login (cookiepath='./koacookie.txt')

Userid: koaadmin
Password: ········
Successfully login as koaadmin


#### Metadata Retrival

Archive class contains methods that send queries to KOA server via TAP interface, and receives a metadata table as the result.  The nominal input is a TAP compliant ADQL query; however for KOA users' convenience, a few simple requests have been provided to search by datatime, position, object name.  These short cut methods construct the query and provide the default output columns for the users.

The following examples query the archive with query_adql, query_datetime, query_position, query_object, and query_creiteria methods; as you can see, cookiepath, format, and maxrec are 'optional' input parameters.  If these parameters are left out, the following are the default:

cookiepath:  no default value, only public data will be accessible,
format    :  ipac ASCII table,
maxrec    :  0, this means all the records are returned.

In [3]:
query = "select koaid, instrume, targname, object, imagetyp, frameno, ra2000 as ra, dec2000 as dec, mjd, to_char(date_obs,'YYYY-MM-DD') as date_obs, ut, elaptime, waveblue, wavered, progid, proginst, progpi, progtitl, sig2nois, slitwidt, slitlen, spatscal, specres, filehand, lower(semid) as semid, propmin, amplist, ampmode, binning, xdispers, deckname, lampqtz2, iodin, iodout, collblue, collred, ccdgain, ccdspeed, echangl, fil1name, fil2name, mosmode, nvidinp, precol, postpix, roqual, window, xdangl, equinox, datlevel, outfile from koa.koa_hires_v6 where (contains(point('J2000',ra,dec), circle('J2000', 77.28659645, -8.75408078, 0.5))=1) order by utdatetime"

arch.query_adql (query, \
    './hires.meta.adql.tbl', \
    cookiepath='./koacookie.txt', \
    format='ipac')

submitting request...
Result downloaded to file [./hires.meta.adql.tbl]


In [4]:
 arch.query_datetime ('hires', \
                  '2019-01-01 00:00:00/2019-03-15 00:00:00', \
                  'hires.meta.datetime.tbl', \
                  cookiepath='./koacookie.txt', \
                  format='ipac', \
                  maxrec='20')


submitting request...
Result downloaded to file [hires.meta.datetime.tbl]


In [5]:
arch.query_position ('hires', \
                  'circle 77.27 -8.748 0.5', \
                  './hires.meta.pos.tbl', \
                  format='ipac')

submitting request...
Result downloaded to file [./hires.meta.pos.tbl]


In [6]:
arch.query_object ('hires', \
                  'hr1679', \
                  './hires.meta.object.tbl', \
                  format='ipac')

resolving object name
object name resolved: ra= 77.286596, dec=-8.754081
submitting request...
Result downloaded to file [./hires.meta.object.tbl]


In [7]:
param = dict()
param['instrument'] = 'hires'
param['datetime'] = '2008-01-01 00:00:00/2008-12-31 00:00:00'
param['pos'] = 'circle 77.28 -8.748 0.5'

arch.query_criteria (param, \
                  './hires.meta.criteria.tbl', \
                  format='ipac')

submitting request...
Result downloaded to file [./hires.meta.criteria.tbl]



#### Data Download

'download' method allows users to download KOA's FITS file and their associated calibration files.  The metadata files must include 'instrume', 'koaid', and 'filehand' columns to enable the download functionality.  These columns are included automatically in the short cut retrival methods: query_datetime, query_position, and query_object; but if you used the query_adql method and constructed the select string yourself, make sure the above columns are in the select statement.

In addtion to the metadata file path and its format (ipac/votable/..), a directory path is required for depositing the returned data.  If the data is proprietary, an optional cookiepath parameter is required; and if you don't wish to download all the files in the metadata table, optional start_row and end_row may be specified in the input parameters. 

##### <font color="#880000">Note: </font>
If a file requested already existed in the directory, it will not be downloaded again.

In [8]:
arch.download ('hires.meta.pos.tbl', \
               'ipac', \
               './dnload_dir', \
               cookiepath='koacookie.txt',
               start_row=0, \
               end_row=1, \
               calibfile=1)                      

Start downloading 2 FITS data you requested;
please check your outdir: ./dnload_dir for  progress.
A total of new 2 FITS files downloaded.
2 new calibration list downloaded.
10 new calibration FITS files downloaded.
